In [4]:
from datasets import load_dataset,load_metric, DownloadConfig

In [21]:
import transformers
print(transformers.__version__)

4.14.1


# loading dataset 加载数据集

In [5]:
raw_datasets = load_dataset("squad")

Using the latest cached version of the module from C:\Users\ChenXiaolong\.cache\huggingface\modules\datasets_modules\datasets\squad\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453 (last modified on Wed Dec 22 09:59:45 2021) since it couldn't be found locally at squad., or remotely on the Hugging Face Hub.
Reusing dataset squad (C:\Users\ChenXiaolong\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [7]:
squad_v2 = False #代表使用的是SQUAD v1 或者 SQUAD v2
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

数据集格式
id编号 answers代表答案 context代表文本片段 question代表问题

In [9]:
raw_datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [10]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

In [17]:
def show_random_elements(dataset,num_example=15):
    assert num_example <= len(dataset),"数据集的长度不够，无法选择足够的示例"
    picks = []
    for _ in range(num_example):
        pick = random.randint(0,len(dataset)-1)
        while pick in picks:
            pick = random.randint(0,len(dataset)-1)
        picks.append(pick)
        
    df = pd.DataFrame(dataset[picks])
    
    for col,type in dataset.features.items():
        if isinstance(type, ClassLabel):
            df[col] = df[col].transform(lambda i:type.names[i])
        elif isinstance(type, Sequence) and isinstance(type.feature, ClassLabel):
            df[col] = df[col].transform(lambda x: [type.feature.names[i] for i in x])
    display(HTML(df.to_html()))
          

In [18]:
show_random_elements(raw_datasets["train"], num_example=15)

,id,title,context,question,answers
0,56e7231d37bdd419002c3d99,Royal_Institute_of_British_Architects,"The content of the acts, particularly section 1 (1) of the amending act of 1938, shows the importance which was then attached to giving architects the responsibility of superintending or supervising the building works of local authorities (for housing and other projects), rather than persons professionally qualified only as municipal or other engineers. By the 1970s another issue had emerged affecting education for qualification and registration for practice as an architect, due to the obligation imposed on the United Kingdom and other European governments to comply with European Union Directives concerning mutual recognition of professional qualifications in favour of equal standards across borders, in furtherance of the policy for a single market of the European Union. This led to proposals for reconstituting ARCUK. Eventually, in the 1990s, before proceeding, the government issued a consultation paper ""Reform of Architects Registration"" (1994). The change of name to ""Architects Registration Board"" was one of the proposals which was later enacted in the Housing Grants, Construction and Regeneration Act 1996 and reenacted as the Architects Act 1997; another was the abolition of the ARCUK Board of Architectural Education.",What was the new name given to ARCUK in the '90s?,"{'text': ['Architects Registration Board'], 'answer_start': [985]}"
1,572866e54b864d1900164995,History_of_India,"The Shunga Empire or Shunga Empire was an ancient Indian dynasty from Magadha that controlled vast areas of the Indian subcontinent from around 187 to 78 BCE. The dynasty was established by Pushyamitra Shunga, after the fall of the Maurya Empire. Its capital was Pataliputra, but later emperors such as Bhagabhadra also held court at Besnagar, modern Vidisha in Eastern Malwa. Pushyamitra Shunga ruled for 36 years and was succeeded by his son Agnimitra. There were ten Shunga rulers. The empire is noted for its numerous wars with both foreign and indigenous powers. They fought battles with the Kalingas, Satavahanas, the Indo-Greeks, and possibly the Panchalas and Mathuras. Art, education, philosophy, and other forms of learning flowered during this period including small terracotta images, larger stone sculptures, and architectural monuments such as the Stupa at Bharhut, and the renowned Great Stupa at Sanchi. The Shunga rulers helped to establish the tradition of royal sponsorship of learning and art. The script used by the empire was a variant of Brahmi and was used to write the Sanskrit language. The Shunga Empire played an imperative role in patronizing Indian culture at a time when some of the most important developments in Hindu thought were taking place.",Who founded the Shuga empire after the fall of the Maurya empire?,"{'text': ['Pushyamitra Shunga'], 'answer_start': [190]}"
2,56d67a8d1c8504140094714d,2008_Sichuan_earthquake,"Malaysia-based Yazhou Zhoukan conducted an interview with former researcher at the China Seismological Bureau Geng Qingguo (耿庆国), in which Geng claimed that a confidential written report was sent to the State Seismological Bureau on April 30, 2008, warning about the possible occurrence of a significant earthquake in Ngawa Prefecture region of Sichuan around May 8, with a range of 10 days before or after the quake. Geng, while acknowledging that earthquake prediction was broadly considered problematic by the scientific community, believed that ""the bigger the earthquake, the easier it is to predict."" Geng had long attempted to establish a correlation between the occurrence of droughts and earthquakes; Premier Zhou Enlai reportedly took an interest in Geng's work. Geng's drought-earthquake correlation theory was first released in 1972, and said to have successfully predicted the 1975 Haicheng and 1976 Tangshan earthquakes. The same Yazhou Zhoukan article pointed out the inherent difficulties associated with predicting

# preprocessing the training data 数据预处理

对输入数据进行tokenize，得到tokens
将tokens转化为预训练模型中需要对应的token ID
将token ID转化为模型需要的输入格式

In [19]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) #distilbert-base-uncased

In [20]:
tokenizer.save_pretrained('./model') #save vocab and so on

('./model\\tokenizer_config.json',
 './model\\special_tokens_map.json',
 './model\\vocab.txt',
 './model\\added_tokens.json',
 './model\\tokenizer.json')

In [22]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [23]:
print("单个文本tokenize: {}".format(tokenizer.tokenize("What is your name?"), add_special_tokens=True))
print("2个文本tokenize: {}".format(tokenizer.tokenize("My name is Sylvain.", add_special_tokens=True)))

单个文本tokenize: ['what', 'is', 'your', 'name', '?']
2个文本tokenize: ['[CLS]', 'my', 'name', 'is', 'sy', '##lva', '##in', '.', '[SEP]']


In [24]:
tokenizer("What is your name?")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

问答数据三元组<question, context, answer>中的超长context截断，那么我们可能丢掉答案（因为我们是从context中抽取出一个小片段作为答案）。为了解决这个问题，下面的代码找到一个超过长度的例子，
如何进行处理。我们把超长的输入切片为多个较短的输入，每个输入都要满足模型最大长度输入要求。由于答案可能存在与切片的地方，因此我们需要允许相邻切片之间有交集，代码中通过doc_stride参数控制。

In [34]:
len(tokenizer(raw_datasets["train"][0]["context"])["input_ids"])

160

In [25]:
for i, example in enumerate(raw_datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = raw_datasets["train"][i]

In [26]:
len(tokenizer(example["question"], example["context"])["input_ids"])

396

In [27]:
max_length = 384 # 输入feature的最大长度，question和context拼接之后
doc_stride = 128 # 2个切片之间的重合token数量。

In [44]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_offsets_mapping=True,#每个token在切片后context的位置和原始超长context里位置的对应关系
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [43]:
for i, x in enumerate(tokenized_example["input_ids"][:2]):
    print("切片: {}".format(i))
    print(tokenizer.decode(x))

切片: 0
[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth a

每个token在切片后context的位置和原始超长context里位置的对应关系,
tokenized_example第0片的前100个tokens在原始context片里的位置。注意第一个token是[CLS]设定为(0, 0)是因为这个token不属于qeustion或者answer的一部分。第2个token对应的起始和结束位置是0和3。我们可以根据切片后的token id转化对应的token；然后使用offset_mapping参数映射回切片前的token位置，找到原始位置的tokens。


In [45]:
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38), (38, 39), (40, 50), (51, 55), (56, 60), (60, 61), (0, 0), (0, 3), (4, 7), (7, 8), (8, 9), (10, 20), (21, 25), (26, 29), (30, 34), (35, 36), (36, 37), (37, 40), (41, 45), (45, 46), (47, 50), (51, 53), (54, 58), (59, 61), (62, 69), (70, 73), (74, 78), (79, 86), (87, 91), (92, 96), (96, 97), (98, 101), (102, 106), (107, 115), (116, 118), (119, 121), (122, 126), (127, 138), (138, 139), (140, 146), (147, 153), (154, 160), (161, 165), (166, 171), (172, 175), (176, 182), (183, 186), (187, 191), (192, 198), (199, 205), (206, 208), (209, 210), (211, 217), (218, 222), (223, 225), (226, 229), (230, 240), (241, 245), (246, 248), (248, 249), (250, 258), (259, 262), (263, 267), (268, 271), (272, 277), (278, 281), (282, 285), (286, 290), (291, 301), (301, 302), (303, 307), (308, 312), (313, 318), (319, 321), (322, 325), (326, 330), (330, 331), (332, 340), (341, 351), (352, 354), (355, 363), (364, 373), (374,

In [47]:
first_token_id = tokenized_example["input_ids"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0],first_token_id)

how 2129


In [50]:
offsets = tokenized_example["offset_mapping"][0][1]
offsets
#print(example["question"][offsets[0]:offsets[1]],offsets)

(0, 3)

In [52]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)
#None对应了special tokens，然后0或者1分表代表第1个文本和第2个文本，由于我们qeustin第1个传入，context第2个传入，所以分别对应question和context。

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [53]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])
print(start_char,end_char)

30 40


In [54]:
# 找到当前文本的Start token index.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1
print(token_start_index)

16


In [56]:
# 找到当前文本的End token idnex.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1
print(token_end_index)

382


In [58]:
# 检测答案是否在文本区间的外部，这种情况下意味着该样本的数据标注在CLS token位置。
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # 将token_start_index和token_end_index移动到answer所在位置的两侧.
    # 注意：答案在最末尾的边界条件.
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print("start_position: {}, end_position: {}".format(start_position, end_position))
else:
    print("The answer is not in this feature.")
    

start_position: 23, end_position: 26


In [59]:
#验证方式是：使用答案所在位置下标，取到对应的token ID，然后转化为文本，然后和原始答案进行但对比。
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

over 1, 600
over 1,600
